In [ ]:
import pandas as pd
import math
def regularizar_nombre(x, base=None, diccionario=None):
    # Caso cero, donde ambos son iguales, se queda el primero
    if x[1] == x[3]:
        return x[1]

    # Primer caso, donde ambos tienen valor, nos quedamos con el más largo
    if (x[1] != x[3]) and pd.notna(x[1]) and pd.notna(x[3]):
        return x[1] if len(x[1]) > len(x[3]) else x[3]

    # Segundo caso, donde solo hay nombre_1, nos quedamos con nombre_1
    if (x[1] != x[3]) and pd.notna(x[1]) and pd.isna(x[3]):
        return x[1]

    # Tercer caso, donde solo hay nombre_2, nos quedamos con nombre_2
    if (x[1] != x[3]) and pd.isna(x[1]) and pd.notna(x[3]):
        return x[3]

    # Cuarto caso, donde no hay data, hay que ir a buscar a la base de datos
    if (x[1] != x[3]) and pd.isna(x[1]) and pd.isna(x[3]):
        if base is not None:
            serie = base.loc[base["Rut"].str.strip() == x[0].strip(), "Nombre"]
            if not serie.empty:
                return serie.iloc[0]

        if diccionario is not None:
            serie = diccionario.loc[diccionario["Rut"].str.strip() == x[0].strip(), "Nombre"]
            if not serie.empty:
                return serie.iloc[0]
            else:
                return "No está"
            
        return "No está"


        # if not es_ant_malo:
        #     serie = ant_malo.loc[ant_malo["rut"] == x[0], "nombre"]
        #     if not serie.empty:
        #         return serie.iloc[0]
        #     else:
        #         serie = base.loc[base["Rut"] == x[0], "Nombre"]
        #         if not serie.empty:
        #             return serie.iloc[0]
        #         else:
        #             return "No está"
        
        


def procesar(nombre, delimiter=",", es_ant_malo=False):
    df = pd.read_csv(nombre, delimiter=delimiter, header=12)
    df = df.iloc[:-1]

    clean_df = df[~df["Vencto."].str.contains("/").fillna(False)].reset_index(drop=True)
    
    # Crea "join_key" para juntar ambas filas por socio más adelante
    clean_df["join_key"] =  pd.Series(clean_df.index).apply(lambda x: math.floor(x/2))
    
    # DataFrame de rut y nombre
    df_principal = clean_df.iloc[::2].reset_index(drop=True)
    df_principal = df_principal[["TP", "Vencto.", "Detalle", "join_key"]]
    df_principal.columns = ['rut', 'nombre_1', 'tipo', 'join_key']
    df_principal = df_principal.set_index("join_key")
    
    # DataFrame de nombre y totales
    df_secundario = clean_df.iloc[1::2].reset_index(drop=True)
    df_secundario = df_secundario[["TP.1", "Unnamed: 8", "Créditos", "Saldo", "Unnamed: 11", "join_key"]]
    df_secundario.columns = ['nombre_2', 'debitos', 'creditos', 'saldo', 'categoria','join_key']
    df_secundario = df_secundario.set_index("join_key")
    
    # Mezcla los 2 dataframes
    df_final = df_principal.join(df_secundario)
    df_final = df_final.reset_index(drop="True")

    base = pd.read_csv("buenos/DiciembreBueno.csv", index_col="Unnamed: 0") # Este es fijo
    diccionario = pd.read_csv("ruts_faltantes.csv") # Este es fijo
    df_final["nombre"] = df_final.apply(regularizar_nombre, axis=1, base=base, diccionario=diccionario)
    return df_final

# Testing

In [104]:
# base = pd.read_csv("buenos/SeptiembreBueno.csv", index_col="Unnamed: 0")
df_final = procesar("malos/202509malo.csv")
# df_final

In [108]:
len(df_final[df_final["nombre"] == "No está"])

0